In [2]:
import sys
sys.path.append('..') 
import os

import psycopg2
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta

# Main

## Data Loading

In [3]:
# """DataLoad"""
# from src.utils.config import get_config
# from src.db_ops.db_ops import PosgreOps

# # Load Config
# config = get_config()
# # Initialize database operator, this case posgre operator
# pgops = PosgreOps(config=config)
# ### Define the required parameters to load the data ###
# # -------------------------------------------------------- #
# months = 84
# table_name = "daas.epdk_petrol_province"
# # -------------------------------------------------------- #
# # Fetch data from database
# epdk_petrol_province_data = pgops.get_monthly_data(table_name=table_name,
#                                                 today=date.today(), months=months)
# # Close the connection after obtaining the data
# pgops.shutdown()

In [4]:
# Load Raw data
epdk_petrol_province_data = pd.read_csv('data/daas_raw_data.csv', index_col=[0])
epdk_petrol_province_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5904 entries, 0 to 5903
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            5904 non-null   object 
 1   province        5904 non-null   object 
 2   gasolene_types  5904 non-null   float64
 3   diesel_types    5904 non-null   float64
 4   fuel_oil_types  5890 non-null   float64
 5   aviation_fuels  5875 non-null   float64
 6   marine_fuels    5826 non-null   float64
 7   paraffin_oil    5838 non-null   float64
 8   other_types     5824 non-null   float64
 9   total           5904 non-null   float64
 10  percentage      5904 non-null   float64
dtypes: float64(9), object(2)
memory usage: 553.5+ KB


In [5]:
epdk_petrol_province_data.query("province == 'İSTANBUL'").drop(columns=["percentage"]).head()

,date,province,gasolene_types,diesel_types,fuel_oil_types,aviation_fuels,marine_fuels,paraffin_oil,other_types,total
0,2021-01-07,İSTANBUL,64029877.00,309312242.0,227.04,35862161.00,150.0,0.00,0.0,4.095813e+08
82,2021-01-06,İSTANBUL,56566776.00,296383914.0,120.82,27658.03,0.0,15.67,0.0,3.807452e+05
164,2021-01-05,İSTANBUL,36589366.00,217554409.0,216.26,16877432.00,0.0,11771.00,13241.0,2.712536e+08
246,2021-01-04,İSTANBUL,42943407.00,261733757.0,334.94,20745681.00,200.0,15738.00,0.0,3.259735e+08
328,2021-01-03,İSTANBUL,51458.42,292829027.0,384.59,22623562.00,370.0,14327.00,0.0,3.676799e+08


## Preprocessing

In [6]:
"""Preprocessing"""
from src.preprocessing.preprocess import PreprocessData

# Initialize preprocess object
prep = PreprocessData(data=epdk_petrol_province_data)
### Define parameters to preprocess ###
# -------------------------------------------------------- #
target_col_list = ["date","province","diesel_types"]
row_drop_dict = {"province": "Toplam"} 
# format_date_flag = True # True by default
col_rename_dict = {"diesel_types":"current_month_consumption"}
anomaly_col = "current_month_consumption" 
# -------------------------------------------------------- #
# Preprocess data with given parameters
preprocessed_df = prep.preprocess_data(target_col_list=target_col_list,
                                 row_drop_dict=row_drop_dict,
                                 col_rename_dict=col_rename_dict,
                                 anomaly_col=anomaly_col)


Missing Dates are:  [numpy.datetime64('2021-10-01T00:00:00.000000000')]


## Feature Engineering

In [7]:
"""FeatureEngineering"""
from src.feature_engineering.feat_eng import FeatureEngineering

# Initialize feature engineering object
feng = FeatureEngineering(data=preprocessed_df)
# Apply feature engineering to the preprocessed data
feature_engineered_df = feng.feature_engineering()

In [ ]:
feature_engineered_df.head()

NameError: name 'feature_engineered_df' is not defined

In [ ]:
from sklearn.preprocessing import minmax_scale, scale
scores = [0.12, 0.12, 0.03, 0.024]
preds = [13, 11, 8, 7]
normalized = minmax_scale(scores)
deneme = np.multiply(preds,normalized)
deneme.sum()/normalized.sum() # this value is expected to be closer to 10

11.878787878787877

In [ ]:
(12*13+12*13+3*8+7*2.4)/(12+12+3+2.4)

12.000000000000002